In [5]:
print('hi')

hi


### Notebook imports

In [6]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import requests
import cv2
import pandas as pd

### Loading the data

In [7]:
(training_images, training_labels), (testing_images, testing_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize image data
training_images = training_images / 255.0
testing_images = testing_images / 255.0

In [8]:
print('Shape of data:')
print('Training images shape: ', training_labels.shape)
print('Training labels shape: ', training_labels.shape)
print('Testing images shape: ', testing_images.shape)
print('Testing labels shape: ', testing_labels.shape)

Shape of data:
Training images shape:  (60000,)
Training labels shape:  (60000,)
Testing images shape:  (10000, 28, 28)
Testing labels shape:  (10000,)


# Creating the model without CNN

#### Model

In [9]:
model_without_cnn = keras.Sequential([

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

#### Compiling the model

In [10]:
model_without_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Fitting the model

In [11]:
print('Fitting the model')
model_without_cnn.fit(training_images, training_labels, epochs=5)

Fitting the model
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5011 - accuracy: 0.8235
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3782 - accuracy: 0.8634
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3378 - accuracy: 0.8777
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3116 - accuracy: 0.8866
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2937 - accuracy: 0.8903


#### Evaluating the model

In [12]:
print('Evaluating the model')
model_without_cnn.evaluate(testing_images, testing_labels)

Evaluating the model
313/313 [==============================] - 1s 2ms/step - loss: 0.3442 - accuracy: 0.8758


[0.3442371189594269, 0.8758000135421753]

#### Make predictions

In [13]:
predictions = model_without_cnn.predict(testing_images[:10])
index = 0

for prediction in predictions:

    print(f'Prediction: {np.argmax(prediction)} \t Actual: {testing_labels[index]}')
    index = index + 1


1/1 [==============================] - 0s 345ms/step
Prediction: 9 	 Actual: 9
Prediction: 2 	 Actual: 2
Prediction: 1 	 Actual: 1
Prediction: 1 	 Actual: 1
Prediction: 6 	 Actual: 6
Prediction: 1 	 Actual: 1
Prediction: 4 	 Actual: 4
Prediction: 6 	 Actual: 6
Prediction: 5 	 Actual: 5
Prediction: 7 	 Actual: 7


#### Going to see if it can detect other images

In [14]:
classes = pd.Series({'tshirt or top': 0, 'trouser':1, 'pullover':2, 'dress':3, 'coat':4, 'sandal':5, 'shirt':6, 'sneaker':7, 'bag':8, 'ankle boot':9})

In [15]:
# Specify the file path of the image
image_path = 'Bag.webp'

# Load the image using OpenCV
image = cv2.imread(image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Resize and preprocess the grayscale image
input_size = (28, 28)  # Example input size from MNIST Fashion dataset
resized_image = cv2.resize(gray_image, input_size)
normalized_image = resized_image / 255.0  # Normalize pixel values

# Convert to tensor and add batch dimension
input_tensor = tf.convert_to_tensor(normalized_image[np.newaxis, ..., np.newaxis])

# Make predictions
predictions = model_without_cnn.predict(input_tensor)
predicted_class = np.argmax(predictions)

# Prediction
print('Predicted class:', predicted_class)
print('Actual class:', classes.loc['bag'])

1/1 [==============================] - 0s 103ms/step
Predicted class: 8
Actual class: 8


# Creating the model with a CNN

#### Building the model

In [16]:
# Improving the model
improved_model = tf.keras.Sequential([

    # Convolution (applying a filter)
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    # Pooling (max value of every pixel)
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Original layers in the other neural network
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])
improved_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 800)               0         
                                                                 
 dense_4 (Dense)             (None, 128)              

#### Compiling the model

In [17]:
improved_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Fitting the model

In [18]:
improved_model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 29s 15ms/step - loss: 0.4596 - accuracy: 0.8330
Epoch 2/5
1875/1875 [==============================] - 31s 17ms/step - loss: 0.3086 - accuracy: 0.8869
Epoch 3/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.2624 - accuracy: 0.9028
Epoch 4/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.2314 - accuracy: 0.9152
Epoch 5/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.2086 - accuracy: 0.9220


In [19]:
print(testing_labels[:100])

[9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0 2 5 7 9 1 4 6 0 9 3 8 8 3 3 8 0 7
 5 7 9 6 1 3 7 6 7 2 1 2 2 4 4 5 8 2 2 8 4 8 0 7 7 8 5 1 1 2 3 9 8 7 0 2 6
 2 3 1 2 8 4 1 8 5 9 5 0 3 2 0 6 5 3 6 7 1 8 0 1 4 2]


# Experimenting with the model

## Editing the number of filters in the convolution

In [20]:
# Editing convolutions
convolutions_edited_model = tf.keras.Sequential([

    # Convolution (applying a filter)
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
    # Pooling (max value of every pixel)
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Original layers in the other neural network
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

convolutions_edited_model.summary()
convolutions_edited_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
convolutions_edited_model.fit(training_images, training_labels, epochs=5)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 16)        2320      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 400)               0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [21]:
convolutions_edited_model.evaluate(testing_images, testing_labels)

313/313 [==============================] - 2s 4ms/step - loss: 0.2983 - accuracy: 0.8886


[0.29826605319976807, 0.8885999917984009]

### Accuracy goes down when the number of filters decreases

## Removing a convolution

In [22]:
# Editing convolutions
convolution_removed_model = tf.keras.Sequential([

    # Convolution (applying a filter)
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    # Pooling (max value of every pixel)
    tf.keras.layers.MaxPooling2D(2, 2),

    # Original layers in the other neural network
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

convolution_removed_model.summary()
convolution_removed_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
convolution_removed_model.fit(training_images, training_labels, epochs=5)
convolution_removed_model.evaluate(testing_images, testing_labels)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 5408)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               692352    
                                                                 
 dense_9 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
________________________________________________

[0.2557075619697571, 0.9110999703407288]